<a href="https://colab.research.google.com/github/kelecheeii/Insurance-Project-3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00


In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import warnings

warnings.filterwarnings('ignore')

In [5]:
# scaling and encoding algorithms
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder

# column transformer
from sklearn.compose import ColumnTransformer, make_column_selector

# handling outliers
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# variance threshold and RFECV
from sklearn.feature_selection import VarianceThreshold, RFECV
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Machine learning algorithms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

# SMOTE oversampling
from imblearn.over_sampling import SMOTE

# Machine Learning - Optuna
import optuna
from optuna.samplers import TPESampler

# Train test split
from sklearn.model_selection import train_test_split

# Pipelines
from imblearn.pipeline import Pipeline

# Machine learning - feature importance

In [6]:
from google.colab import files

uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


In [7]:
# Load the training data from 'train.csv' into a DataFrame
training_data = pd.read_csv('train.csv')

# Load the testing data from 'test.csv' into a DataFrame
testing_data = pd.read_csv('test.csv')

In [8]:
# Creating a copy of my dataset

training_data_copy = training_data.copy()

In [9]:
# Display the summary information of the training dataset

training_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         58592 non-null  object 
 1   policy_tenure                     58592 non-null  float64
 2   age_of_car                        58592 non-null  float64
 3   age_of_policyholder               58592 non-null  float64
 4   area_cluster                      58592 non-null  object 
 5   population_density                58592 non-null  int64  
 6   make                              58592 non-null  int64  
 7   segment                           58592 non-null  object 
 8   model                             58592 non-null  object 
 9   fuel_type                         58592 non-null  object 
 10  max_torque                        58592 non-null  object 
 11  max_power                         58592 non-null  object 
 12  engi

In [10]:
# Create DataFrame from the dictionary
df = pd.DataFrame(training_data_copy)

# Apply extraction logic to the entire column using regular expression
df[['max_torque_nm', 'max_torque_rpm']] = df['max_torque'].str.extract(r'(\d+)Nm@(\d+)rpm')

# Convert the extracted values to integers
df['max_torque_nm'] = df['max_torque_nm'].astype(int)
df['max_torque_rpm'] = df['max_torque_rpm'].astype(int)

# Optionally, drop the original 'max_torque' column
df.drop(columns=['max_torque'], inplace=True)


In [11]:
# Extract `bhp` and `rpm` values
df[['max_power_bhp', 'max_power_rpm']] = df['max_power'].str.extract(r'([\d.]+)bhp@(\d+)rpm')

# Convert to appropriate data types
df['max_power_bhp'] = df['max_power_bhp'].astype(float)  # Decimals in bhp
df['max_power_rpm'] = df['max_power_rpm'].astype(int)    # Integers for rpm

# Optionally drop the original column
df.drop(columns=['max_power'], inplace=True)

In [12]:
train_copy = df

In [13]:
# Print the resulting DataFrame to verify
train_copy.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim,max_torque_nm,max_torque_rpm,max_power_bhp,max_power_rpm
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,0,0,60,3500,40.36,6000
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,0,0,60,3500,40.36,6000
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,0,0,60,3500,40.36,6000
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,2,0,113,4400,88.50,6000
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,2,0,91,4250,67.06,5500


In [14]:
# Identifying boolean-like columns in the dataset
# A boolean-like column is defined as one with only 2 unique values.
bool_val = [col for col in train_copy.columns if train_copy[col].nunique() <= 2]

# Display the values in those columns
train_copy[bool_val].head()

,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,rear_brakes_type,cylinder,transmission_type,gear_box,is_front_fog_lights,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,is_claim
0,No,No,No,Yes,No,Drum,3,Manual,5,No,...,No,No,No,No,Yes,No,No,No,Yes,0
1,No,No,No,Yes,No,Drum,3,Manual,5,No,...,No,No,No,No,Yes,No,No,No,Yes,0
2,No,No,No,Yes,No,Drum,3,Manual,5,No,...,No,No,No,No,Yes,No,No,No,Yes,0
3,Yes,Yes,No,Yes,Yes,Drum,4,Automatic,5,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,0
4,No,No,No,No,Yes,Drum,3,Automatic,5,No,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,0


In [15]:
# Define the target column
target_col = 'is_claim'

# Initialize a dictionary to store label mappings for each column
label_mappings = {}

# Apply LabelEncoder to each column in bool_val
for col in bool_val:
    if col != target_col:  # Exclude the target column
        label_encoder = LabelEncoder()
        train_copy[col] = label_encoder.fit_transform(train_copy[col])
        # Store the label mappings for this column
        label_mappings[col] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the label mappings for all columns
for col, mapping in label_mappings.items():
    print(f"Column: {col}, Mapping: {mapping}")

Column: is_esc, Mapping: {'No': 0, 'Yes': 1}
Column: is_adjustable_steering, Mapping: {'No': 0, 'Yes': 1}
Column: is_tpms, Mapping: {'No': 0, 'Yes': 1}
Column: is_parking_sensors, Mapping: {'No': 0, 'Yes': 1}
Column: is_parking_camera, Mapping: {'No': 0, 'Yes': 1}
Column: rear_brakes_type, Mapping: {'Disc': 0, 'Drum': 1}
Column: cylinder, Mapping: {3: 0, 4: 1}
Column: transmission_type, Mapping: {'Automatic': 0, 'Manual': 1}
Column: gear_box, Mapping: {5: 0, 6: 1}
Column: is_front_fog_lights, Mapping: {'No': 0, 'Yes': 1}
Column: is_rear_window_wiper, Mapping: {'No': 0, 'Yes': 1}
Column: is_rear_window_washer, Mapping: {'No': 0, 'Yes': 1}
Column: is_rear_window_defogger, Mapping: {'No': 0, 'Yes': 1}
Column: is_brake_assist, Mapping: {'No': 0, 'Yes': 1}
Column: is_power_door_locks, Mapping: {'No': 0, 'Yes': 1}
Column: is_central_locking, Mapping: {'No': 0, 'Yes': 1}
Column: is_power_steering, Mapping: {'No': 0, 'Yes': 1}
Column: is_driver_seat_height_adjustable, Mapping: {'No': 0, 'Yes':

In [16]:
train_copy[bool_val].head()

,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,rear_brakes_type,cylinder,transmission_type,gear_box,is_front_fog_lights,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,is_claim
0,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1,1,0,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,1,1,1,0,1,1,1,0


In [17]:
# Separating the dataset into features (X) and target variable (y)
X = train_copy.drop('is_claim', axis=1)
y = train_copy['is_claim']

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         58592 non-null  object 
 1   policy_tenure                     58592 non-null  float64
 2   age_of_car                        58592 non-null  float64
 3   age_of_policyholder               58592 non-null  float64
 4   area_cluster                      58592 non-null  object 
 5   population_density                58592 non-null  int64  
 6   make                              58592 non-null  int64  
 7   segment                           58592 non-null  object 
 8   model                             58592 non-null  object 
 9   fuel_type                         58592 non-null  object 
 10  engine_type                       58592 non-null  object 
 11  airbags                           58592 non-null  int64  
 12  is_e

In [19]:
# Splitting the dataset into training and testing sets
# Allocating 30% of the data for testing, Setting a random seed for reproducibility.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=47)

In [21]:
# defining the column groups

numeric_columns = [
    'policy_tenure', 'age_of_car', 'age_of_policyholder',
    'population_density', 'airbags', 'displacement', 'cylinder',
    'gear_box', 'turning_radius', 'length', 'width', 'height',
    'gross_weight', 'ncap_rating', 'max_torque_nm', 'max_torque_rpm',
    'max_power_bhp', 'max_power_rpm','rear_brakes_type','make','is_esc',
    'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
    'is_parking_camera', 'is_front_fog_lights', 'is_rear_window_wiper',
    'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist',
    'is_power_door_locks', 'is_central_locking', 'is_power_steering',
    'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
    'is_ecw', 'is_speed_alert']

ordinal_columns = [
    'segment',]

nominal_columns = [
    'policy_id', 'area_cluster','model',
    'fuel_type', 'engine_type', 'steering_type','transmission_type']

# Define the preprocessing pipeline for different types of columns
# Handle ordinal features by applying OrdinalEncoder (encoding them as integer values)
# Handle nominal features using OneHotEncoder (creating dummy variables for each category)
# We set 'handle_unknown='ignore'' to ignore unseen categories during transformation
# 'sparse_output=True' ensures that the output is in sparse matrix format
# Scale numerical features using StandardScaler to ensure all numeric features have zero mean and unit variance

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), ordinal_columns),
        ('nominal', OneHotEncoder(handle_unknown='ignore', sparse_output=True), nominal_columns),
        ('numeric', StandardScaler(), numeric_columns)])



In [22]:
# Preprocess the data using the 'preprocessor' pipeline
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)



In [32]:
# instatiating the LGBM classifer for feature selection
lightgbm_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, verbose=-1)

In [66]:
# Fit the model to evaluate feature importance
lightgbm_model.fit(X_train_preprocessed, y_train)

# Get feature importance
importances = lightgbm_model.feature_importances_

# Get indices of the most important features
important_features = importances.argsort()[-100:]  # Select top 100 important features

# Create a new dataset with only the most important features
X_train_selected = X_train_preprocessed[:, important_features]
X_test_selected = X_test_preprocessed[:, important_features]

print(f"Shape after feature selection: {X_train_selected.shape}")
print(f"Shape after feature selection: {X_test_selected.shape}")


Shape after feature selection: (41014, 100)
Shape after feature selection: (17578, 100)


In [69]:
# Apply SMOTE separately before fitting the pipeline
smote = SMOTE(random_state=47)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected, y_train)


In [70]:
# Step 1: Define the objective function for Optuna
def objective(trial):
    # Hyperparameters for LGBMClassifier
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    num_leaves = trial.suggest_int('num_leaves', 2, 15)
    max_depth = trial.suggest_int('max_depth', 3, 10)


    # Initialize the LGBMClassifier with hyperparameters
    model = LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        max_depth=max_depth,
        random_state=47
    )

    # Fit the model with training data
    model.fit(X_train_resampled, y_train_resampled)

    # Predict probabilities for the test set
    y_pred_proba = model.predict_proba(X_test_selected)[:, 1]

    # Return the ROC AUC score as the objective value
    return roc_auc_score(y_test, y_pred_proba)



In [71]:
# Step 2: Set up the Optuna study
study = optuna.create_study(direction='maximize', sampler= TPESampler())  # We aim to maximize the ROC AUC score

# Step 3: Run the optimization over 50 trials
study.optimize(objective, n_trials=50)



[I 2025-01-21 08:38:24,682] A new study created in memory with name: no-name-e3f0368d-2460-446f-81e2-158a1742a3e4
[I 2025-01-21 08:38:29,916] Trial 0 finished with value: 0.6260966354166712 and parameters: {'n_estimators': 263, 'learning_rate': 0.10516828494433153, 'num_leaves': 5, 'max_depth': 5}. Best is trial 0 with value: 0.6260966354166712.
[I 2025-01-21 08:38:33,705] Trial 1 finished with value: 0.6240780941323747 and parameters: {'n_estimators': 194, 'learning_rate': 0.028919446424693365, 'num_leaves': 9, 'max_depth': 9}. Best is trial 0 with value: 0.6260966354166712.
[I 2025-01-21 08:38:38,486] Trial 2 finished with value: 0.6212334603058153 and parameters: {'n_estimators': 455, 'learning_rate': 0.24245779313042623, 'num_leaves': 8, 'max_depth': 9}. Best is trial 0 with value: 0.6260966354166712.
[I 2025-01-21 08:38:42,316] Trial 3 finished with value: 0.6216074134425015 and parameters: {'n_estimators': 415, 'learning_rate': 0.2596914239851214, 'num_leaves': 4, 'max_depth': 6}

In [72]:
# Step 4: Retrieve and print the best trial
best_trial = study.best_trial
print(f"Best parameters: {best_trial.params}")
print(f"Best ROC AUC score: {best_trial.value}")




Best parameters: {'n_estimators': 230, 'learning_rate': 0.19231431698143225, 'num_leaves': 3, 'max_depth': 4}
Best ROC AUC score: 0.6307884063280916


In [73]:
# Step 5: Get the best model and train it on the selected features
best_model = LGBMClassifier(
    n_estimators=best_trial.params['n_estimators'],
    learning_rate=best_trial.params['learning_rate'],
    num_leaves=best_trial.params['num_leaves'],
    max_depth=best_trial.params['max_depth'],
    random_state=47
)

# Train the best model
best_model.fit(X_train_resampled, y_train_resampled)

# Step 6: Evaluate the model on the test set
y_pred_proba = best_model.predict_proba(X_test_selected)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test ROC AUC Score: {roc_auc}")

Test ROC AUC Score: 0.6307884063280916


In [81]:
# Predict probabilities on the test set
y_prob = best_model.predict_proba(X_test_selected)[:, 1]

# Define a threshold for classification
threshold = 0.296  # Default threshold, can be tuned for better performance
y_pred = (y_prob >= threshold).astype(int)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)  # Handle cases with no positive predictions
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
confusion_mat = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(confusion_mat)

Accuracy: 0.5988
Precision: 0.0905
Recall: 0.5827
F1 Score: 0.1566
ROC AUC Score: 0.6308
[[9870 6584]
 [ 469  655]]
